In [33]:
R.<x,y> = PolynomialRing(RR, 2, "xy", order = "lex")

def div_poly(F, f, R = R):
    """
    Implementa el algoritmo de division
    de polinomios multivariados.
    De: Ideals, Varieties, and Algorithms, pag 65
    F es una lista de polinomios, f es el polinomio a dividir
    R es el conjunto de polinomios sobre el que se
    """
    q = [R(0) for _ in range(len(F))]
    r = R(0)
    p = f

    while p!=R(0):
        i=0
        divisionoccurred = False
        while i<len(F) and not divisionoccurred:

            if hasattr(R, "monomial_quotient") and R.base_ring() == QQ:
                quo = R.monomial_quotient(p.lt(),F[i].lt(),coeff=True)
                rem = p.lt().reduce(F[i].lt())
            else:
                quo, rem = p.lt().quo_rem(F[i].lt())
            if rem == 0:
                q[i] = q[i]+quo
                p = p - quo*F[i]
                divisionoccurred = True
            else:
                i += 1
        if not divisionoccurred:
            r = r+p.lt()
            p = p-p.lt()

    return q, r

def poly_to_Macaulay(F, D):
    R = F[0].parent()
    order = R.term_order()
    
    # Paso 1: Construimos todos los polinomios u*f
    UF = []
    for f in F:
        df = f.degree()
        for d in range(D - df + 1):
            for u in R.monomials_of_degree(d):
                UF.append(u * f)

    # Paso 2: Construimos  el conjunto U de monomios que aparecen
    U = set()
    for g in UF:
        for m in g.monomials():
            U.add(m)

    # Paso 3: Ordenamos U con el orden monomial del anillo
    U = sorted(U, reverse=True)
    print(U)

    # Paso 4: Construimos la matriz de Macaulay
    M = []
    for g in UF:
        fila = [g.monomial_coefficient(u) for u in U]
        M.append(fila)

    return Matrix(M), U



def macaulay_to_Poly(M, U):
    
    R = U[0].parent()
    polinomios_base = []
    
    for fila in M.rows():
        # Sage: M.rows() ya maneja la conversión de la matriz a una secuencia de filas
        
        if not fila.is_zero():
            # Construir el polinomio sumando el producto (coeficiente * monomio)
            # zip(fila, U) empareja el coeficiente de la columna j con el monomio U[j]
            p = sum(c * m for c, m in zip(fila, U))
            
            # Es buena práctica asegurar que el coeficiente líder sea 1 si M es RREF
            if p.lc() != 0 and p.lc() != 1:
                 p = p / p.lc()
                 
            polinomios_base.append(p)
            
    # Devolvemos solo los polinomios reconstruidos
    return polinomios_base


def find_Grobner_Basis(F):
    grados = [f.degree() for f in F]
    D = max(grados)
    basis = F
    
    entered_loop = False
    
    while is_Groebner(basis) == False: 
        entered_loop = True
        M, U = poly_to_Macaulay(F,D)
        M_echelon = M.echelon_form()
        basis = macaulay_to_Poly(M_echelon, U)
        D += 1
        
    if entered_loop:    
        return basis, D-1
    else:
        return basis, D



# Implementando el ejemplo 5 del libro:
Sean $f_1=xy-1$, $f_2=y^2-1$, con órden lexicográfico. Si dividimos $f=xy^2-x$ por $F=(f_1,f_2)$ obtenemos:
$$
xy^2-x=y \cdot (xy-1) + 0\cdot (y^2-1)+(-x+y)
$$

Si tomamos $F=(f_2,f_1)$, obtenemos:
$$
xy^2-x= x\cdot (y^2-1) + 0 \cdot(xy-1) + 0
$$


In [28]:
R.<x,y> = PolynomialRing(RR, 2, "xy", order = "lex")

f = x*y^2-x; f1=x*y-1; f2=y^2-1

[q1,q2], r = div_poly([f1,f2],f)

print("Caso 1:")
show(html(f"${f} = {q1} ({f1}) + {q2} ({f2}) + ({r})$"))


print("Caso 2:")
f = x*y^2-x
f1=x*y-1
f2=y^2-1

[q2,q1], r = div_poly([f2,f1],f)
show(html(f"${f} = {q2} ({f2}) + {q1} ({f1}) + {r}$"))



Caso 1:


\(x*y^2 - x = y (x*y - 1.00000000000000) + 0 (y^2 - 1.00000000000000) + (-x + y)\)

Caso 2:


\(x*y^2 - x = x (y^2 - 1.00000000000000) + 0 (x*y - 1.00000000000000) + 0\)

# Aplicando el algoritmo con polinomios aleatorios

In [29]:
R.<x_1,x_2,x_3, x_4> = PolynomialRing(GF(7), order="lex")
f = R.random_element(degree=2, terms = 4)


F = [R.random_element(degree=1, terms=2) for _ in range(3)]

Q, r = div_poly(F, f, R = R)

show(html(f"""
$$
\\begin{{align*}}
f = &{f} \\\\
\\sum_i q_i f_i +r = &{
    sum([q*f for q,f in zip(Q,F)]) + r
    }\\\\
    f_1 = &{F[0]} \\quad \\quad q_1 = {Q[0]} \\\\
    f_2 = &{F[1]} \\quad \\quad q_2 = {Q[1]} \\\\
    f_3 = &{F[2]} \\quad \\quad q_3 = {Q[2]} \\\\
    r = &{r}
\\end{{align*}}
$$
"""))


\[ \begin{align*} f = &-2*x_1 - x_2*x_4 + 3*x_3*x_4 \\ \sum_i q_i f_i +r = &-2*x_1 - x_2*x_4 + 3*x_3*x_4\\     f_1 = &-x_4 - 1 \quad \quad q_1 = x_2 - 3*x_3 \\     f_2 = &-2*x_4 - 1 \quad \quad q_2 = 0 \\     f_3 = &x_2 + x_3 \quad \quad q_3 = 1 \\     r = &-2*x_1 + 3*x_3 \end{align*}\]

# Base escalonada

In [83]:
degree = 3
dim = 3

# Anillo de polinomios que vamos a trabajar
R.<x,y> = PolynomialRing(QQ, order = "lex")
# Ideal de polinomios
I = ideal([R.random_element(degree=10) for _ in range(3)])
# Base de groebner
base = I.groebner_basis()
base

[x^7, x^4*y + 20*y^2, x*y^2, y^3]

In [84]:
# Sorteando la base según el lt de cada polinomio
base = sorted(base, key = lambda x: x.lt())

# Añadiendo el primer elemento
L = [base[0]]

# Añadiendo los polinomios si son 0 módulo los otros elementos
for b in base:
    if b!=R(0) and b.reduce(L) != 0:
        L.append(b)

f = ideal(L).random_element(degree=3)
Q, r = div_poly(L, f)


print(latex(f))
print(latex(Q))

show(html(f"""
$$
\\begin{{align*}}
f = &{f} \\\\
\\sum_i q_i f_i +r = &{
    sum([q*f for q,f in zip(Q,L)]) + r
    }\\\\
    f_1 = &{L[0]} \\quad \\quad q_1 = {Q[0]} \\\\
    f_2 = &{L[1]} \\quad \\quad q_2 = {Q[1]} \\\\
    f_3 = &{L[2]} \\quad \\quad q_3 = {Q[2]} \\\\
    r = &{r}
\\end{{align*}}
$$
"""))


-x y^{2} + \frac{1}{8} y^{3}
\left[0.125000000000000, -1.00000000000000, 0, 0\right]


\[ \begin{align*} f = &-x*y^2 + 1/8*y^3 \\ \sum_i q_i f_i +r = &-x*y^2 + 0.125000000000000*y^3\\     f_1 = &y^3 \quad \quad q_1 = 0.125000000000000 \\     f_2 = &x*y^2 \quad \quad q_2 = -1.00000000000000 \\     f_3 = &x^4*y + 20*y^2 \quad \quad q_3 = 0 \\     r = &0 \end{align*}\]

In [86]:
f = R.random_element(degree=3)
Q, r = div_poly(L, f)



show(html(f"""
$$
\\begin{{align*}}
f = &{f} \\\\
\\sum_i q_i f_i +r = &{
    sum([q*f for q,f in zip(Q,L)]) + r
    }\\\\
    f_1 = &{L[0]} \\quad \\quad q_1 = {Q[0]} \\\\
    f_2 = &{L[1]} \\quad \\quad q_2 = {Q[1]} \\\\
    f_3 = &{L[2]} \\quad \\quad q_3 = {Q[2]} \\\\
    r = &{r}
\\end{{align*}}
$$
"""))


\[ \begin{align*} f = &-1/2*x^2*y + x*y - 1/2*y^3 + 10/7*y^2 \\ \sum_i q_i f_i +r = &-0.500000000000000*x^2*y + x*y - 0.500000000000000*y^3 + 1.42857142857143*y^2\\     f_1 = &y^3 \quad \quad q_1 = -0.500000000000000 \\     f_2 = &x*y^2 \quad \quad q_2 = 0 \\     f_3 = &x^4*y + 20*y^2 \quad \quad q_3 = 0 \\     r = &-0.500000000000000*x^2*y + x*y + 1.42857142857143*y^2 \end{align*}\]